In [25]:
import os
from fastembed import TextEmbedding
from typing import List
from supabase import Client, create_client

In [26]:
def new_supabase_client():
    url: str = os.environ.get("SUPABASE_URL")
    key: str = os.environ.get("SUPABASE_KEY")
    
    supabase: Client = create_client(url, key)

    return supabase

In [27]:
# Gerar os embeddings de consultas
def get_embedding(text: str, embedding_model_id: str) -> List[float]:
    try:
        embedding_model = TextEmbedding(embedding_model_id)
        embeddings = list(embedding_model.passage_embed([text]))
        return embeddings[0].tolist()
    except Exception as e:
        print(f"Erro ao gerar embedding: {e}")
        return [0] * 768

In [29]:
def retrieval_relvante_context(user_query: str, embedding_model_id: str):
    query_embedding = get_embedding(user_query, embedding_model_id)

    supabase = new_supabase_client()

    result = supabase.rpc(
        'match_reports_crm',
        params= {
            'query_embedding': query_embedding,
            'match_count': 5
        }
    ).execute()

    if not result.data:
        return "Nenhum dado relevante encontrado."
    
    context = []

    for doc in result.data:
        chunk = f"""
        # {doc['metadata']['source']} - Chunk {doc['metadata']['chunk_index']}
        # {doc['content']}
        # """

        context.append(chunk)

    return "\n\n---\n\n".join(context)

In [30]:
EMBED_MODEL_ID = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

input_user = 'Quais os principais geradores de oportunidade?'

context = retrieval_relvante_context(user_query=input_user, embedding_model_id=EMBED_MODEL_ID)

print(context)

C:\Users\viva_\AppData\Local\Temp\ipykernel_13540\4176236153.py:4: UserWarning: The model sentence-transformers/paraphrase-multilingual-mpnet-base-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  embedding_model = TextEmbedding(embedding_model_id)



        # 2024.12 - RelatorioMensal.pdf - Chunk 9
        # ```
Melhores Fontes - Criação 1. Indicação por Parceiros: 2 2. Feiras e Eventos: 1
```
        # 

---


        # 2024.05 - RelatorioMensal.pdf - Chunk 10
        # ```
Melhores Fontes - Criação 1. Indicação por Parceiros: 2 2. Feiras e Eventos: 1
```
        # 

---


        # 2023.08 - RelatorioMensal.pdf - Chunk 11
        # ```
Melhores Fontes - Criação 1. Indicação por Parceiros: 4 2. Redes Sociais: 2
```
3. Feiras e Eventos: 1
        # 

---


        # 2021.11.08 - RelatorioSemanal.pdf - Chunk 8
        # Não existem oportunidades perdidas
        # 

---


        # 2021.11.21 - RelatorioSemanal.pdf - Chunk 7
        # Não existem oportunidades perdidas
        # 
